## Búsqueda binaria con índice invertido (BSII)

Importamos librerías de utilidad.

In [ ]:

from pathlib import Path
from typing import List


Importamos nuestras funciones de preprocesamiento de documentos


In [ ]:
from documents import load_docs, Document

In [ ]:
all_docs = load_docs(Path("./data/docs-raw-texts"))

A continuación la definición de la clase que representará al **Índice Invertido (BSII)**. Representa los listings en un diccionario, que internamente ya se encarga del hashing.

In [ ]:
class BSII:
    def __init__(self, docs: List[Document]):
        """
        Inicializa el índice invertido utilizando una lista de documentos.

        Args:
            docs (List[Document]): Lista de objetos Document a indexar.
        """
        self.docs = docs
        self.inverse_index = {}

        # Construir el índice invertido
        for doc in self.docs:

            # Procesa cada término del documento
            for term in doc.term_counts.index:

                # Si el término no está en el índice invertido, lo agrega
                if term not in self.inverse_index:
                    self.inverse_index[term] = set()

                # Añade el documento al listing del término
                self.inverse_index[term].add(doc)

    def search(self, query_document: Document = None, excluded_query_document: Document = None) -> set:
        """
        Realiza una búsqueda en el índice invertido, incluyendo o excluyendo documentos según los términos. Note que procesamos los queries como objetos de tipo Documento.

        Args:
            query_document (Document, opcional): Documento cuyas palabras clave se utilizarán para buscar documentos relevantes. Esto implementa el AND.
            excluded_query_document (Document, opcional): Documento cuyas palabras clave se utilizarán para excluir documentos de los resultados. Esto es equivalente a un NOT.

        Returns:
            set: Un conjunto de documentos que cumplen con los criterios de la búsqueda.
        """
        relevant_docs = set(self.docs)

        # Incluir documentos relevantes según el query_document
        if query_document is not None:
            for term in query_document.term_counts.index:
                if term in self.inverse_index:
                    relevant_docs.intersection_update(self.inverse_index[term])

        # Excluir documentos según el excluded_query_document
        if excluded_query_document is not None:
            for term in excluded_query_document.term_counts.index:
                if term in self.inverse_index:
                    relevant_docs.difference_update(self.inverse_index[term])

        return relevant_docs

    def evaluate_search(self, queries: List[Document], output_path: Path):
        """
        Evalúa las consultas de búsqueda y escribe los resultados en un archivo de salida.

        Args:
            queries (List[Document]): Lista de documentos a utilizar como consultas.
            output_path (Path): Ruta del archivo donde se guardarán los resultados.
        """
        with open(output_path, 'w') as output_file:
            for query in queries:
                relevant_docs = self.search(query_document=query)
                # Escribir el nombre de la consulta seguido de los nombres de los documentos relevantes
                output_file.write(f"{query.name}\t{','.join(doc.name for doc in relevant_docs)}\n")

Construimos nuestra clase con nuestros documentos.

In [ ]:
bsii = BSII(all_docs)

Y calculamos todos los queries, almacenándolos en el formato especificado.

In [ ]:
all_queries = load_docs(Path("./data/queries-raw-texts"))
bsii.evaluate_search(all_queries, 'data/BSII-AND-queries_result')

Qué hacemos con términos que no están en el índice invertido para las búsquedas? Por defecto ahora los está ignorando simplemente:

In [ ]:
bsii.search(query_document=Document('Physiology Esternocleidomastoideo'), excluded_query_document=Document('Swiss'))